## Solutions for the 01_Max-Cut-with-QAOA.ipynb

In [ ]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# Exercise 1 Solution

# Create a list of all the subsets of vertices of sampleGraph
subsets = [[]]
for u in sampleGraph.nodes():
    for i in range(len(subsets)):
        subsets += [subsets[i]+[u]]


# Cycle through the subsets to identify the max cut of the sampleGraph

max_cut_value = 0
max_cut_edges = []
for subset in subsets:
    subset_cut_value = 0
    subset_cut_edges = []
    for u,v in sampleGraph.edges():
        if ((u in subset) and (v not in subset)) or ((v in subset) and (u not in subset)):
            subset_cut_value+=1
            subset_cut_edges.append((u,v))
    if subset_cut_value > max_cut_value:
        max_cut_value = subset_cut_value
        max_cut_edges = subset_cut_edges
        V0 = subset


# Find the complement of V0
V1 = []
for u in sampleGraph.nodes():
    if u not in V0:
        V1.append(u)

print('The',max_cut_value,'edges that make up a max cut of sampleGraph are',max_cut_edges)
print('The lists',V0,'and',V1,'partition the sampleGraph into a the max cut.' )

In [ ]:
# Exercise 2 Solution

# Define a function to generate the Hamiltonian for a max cut problem using the graph G

def hamiltonian_max_cut(sources : List[int], targets : List[int]):
    """Hamiltonian for finding the max cut for the graph  with edges defined by the pairs generated by source and target edges

    Parameters
    ----------
    sources: List[int]
        list of the source vertices for edges in the graph
    targets: List[int]
        list of the target vertices for the edges in the graph

    Returns
    -------
    cudaq.SpinOperator
        Hamiltonian for finding the max cut of the graph defined by the given edges
    """
    hamiltonian = 0
    # Since our vertices may not be a list from 0 to n, or may not even be integers,

    for i in range(len(sources)):
        # Add a term to the Hamiltonian for the edge (u,v)
        qubitu = sources[i]
        qubitv = targets[i]
        hamiltonian += 0.5*(spin.z(qubitu)*spin.z(qubitv)-spin.i(qubitu)*spin.i(qubitv))

    return hamiltonian

In [ ]:
# Exercise 3 Solution
# Problem Kernel

@cudaq.kernel
def qaoaProblem(qubit_0 : cudaq.qubit, qubit_1 : cudaq.qubit, alpha : float):
    """Build the QAOA gate sequence between two qubits that represent an edge of the graph
    Parameters
    ----------
    qubit_0: cudaq.qubit
        Qubit representing the first vertex of an edge
    qubit_1: cudaq.qubit
        Qubit representing the second vertex of an edge
    alpha: float
        Free variable

    """
    x.ctrl(qubit_0, qubit_1)
    rz(2.0*alpha, qubit_1)
    x.ctrl(qubit_0, qubit_1)

In [ ]:
# Exericse 4 Solution
# Mixer Kernel

@cudaq.kernel
def qaoaMixer(qubit_0 : cudaq.qubit, beta : float):
    """Build the QAOA gate sequence that is applied to each qubit in the mixer portion of the circuit
    Parameters
    ----------
    qubit_0: cudaq.qubit
        Qubit
    beta: float
        Free variable

    """
    rx(2.0*beta, qubit_0)


In [ ]:
# Execise 5 Solution

# We now define the kernel_qaoa function which will be the QAOA circuit for our graph
# Since the QAOA circuit for max cut depends on the structure of the graph,
# we'll feed in global concrete variable values into the kernel_qaoa function for the qubit_count, layer_count, edges_src, edges_tgt.
# The types for these variables are restricted to Quake Values (e.g. qubit, int, List[int], ...)
# The thetas plaeholder will be our free parameters (the alphas and betas in the circuit diagrams depicted above)
@cudaq.kernel
def kernel_qaoa(qubit_count :int, layer_count: int, edges_src: List[int], edges_tgt: List[int], thetas : List[float]):
    """Build the QAOA circuit for max cut of the graph with given edges and nodes
    Parameters
    ----------
    qubit_count: int
        Number of qubits in the circuit, which is the same as the number of nodes in our graph
    layer_count : int
        Number of layers in the QAOA kernel
    edges_src: List[int]
        List of the first (source) node listed in each edge of the graph, when the edges of the graph are listed as pairs of nodes
    edges_tgt: List[int]
        List of the second (target) node listed in each edge of the graph, when the edges of the graph are listed as pairs of nodes
    thetas: List[float]
        Free variables to be optimized


    """
    # Let's allocate the qubits
    qreg = cudaq.qvector(qubit_count)

    # And then place the qubits in superposition
    h(qreg)

    # Each layer has two components: the problem kernel and the mixer
    for i in range(layer_count):
        # Add the problem kernel to each layer
        for edge in range(len(edges_src)):
            qubitu = edges_src[edge]
            qubitv = edges_tgt[edge]
            qaoaProblem(qreg[qubitu], qreg[qubitv], thetas[i])
        # Add the mixer kernel to each layer
        for j in range(qubit_count):
            qaoaMixer(qreg[j],thetas[i+layer_count])

In [ ]:
# Exercise 6 Solution

layer_count =1
parameter_count: int = 2 * layer_count
shots =5000

# Set seeds for reproducibility, since different seeds may lead to different max cut solutions
seed={}
for key in subGraph_dictionary:
    i = 4
    seed[key] = i
    i +=100

results = {}

for key in subGraph_dictionary:
    # Set initial parameters for each subgraph
    print('The results for',key,':')
    np.random.seed(seed[key])

    initial_parameters = np.random.uniform(-np.pi, np.pi,
                                                    parameter_count)
    G = subGraph_dictionary[key]
    results[key]= qaoa_for_graph(G, layer_count, shots, seed[key])